In [1]:
import pandas as pd
import requests
from datetime import datetime
from datetime import date


In [2]:
query = open('main_query.txt', 'r').read()
response_data = []
github_token = open('env', 'r').read()
headers = { 'Authorization': f'bearer {github_token}' }

In [3]:
def makeRequest(query):
    print('Start making query')
    page = 0
    previous_cursor = 'null'
    while(page < 10):        
        print(f'Current page {page}')
        response = requests.post('https://api.github.com/graphql', headers=headers, json={'query':query})
        if response.status_code == 200:
            response_json = response.json()['data']['search']
            nodes = response_json['nodes']
            end_cursor = response_json['pageInfo']['endCursor']
            query = query.replace(previous_cursor, f'"{end_cursor}"')
            previous_cursor = f'"{end_cursor}"'
            print(previous_cursor)
            for item in nodes:
                response_data.append(item)
            print(f'Quantity of total nodes: {len(response_data)}')
            page += 1
        else:
            print(f'Some error has happened, status code:  {response.status_code}')
   

       
        
makeRequest(query)

Start making query
Current page 0
Some error has happened, status code:  f502
Current page 0
"Y3Vyc29yOjEwMA=="
Quantity of total nodes: 100
Current page 1
"Y3Vyc29yOjIwMA=="
Quantity of total nodes: 200
Current page 2
"Y3Vyc29yOjMwMA=="
Quantity of total nodes: 300
Current page 3
"Y3Vyc29yOjQwMA=="
Quantity of total nodes: 400
Current page 4
"Y3Vyc29yOjUwMA=="
Quantity of total nodes: 500
Current page 5
"Y3Vyc29yOjYwMA=="
Quantity of total nodes: 600
Current page 6
"Y3Vyc29yOjcwMA=="
Quantity of total nodes: 700
Current page 7
"Y3Vyc29yOjgwMA=="
Quantity of total nodes: 800
Current page 8
"Y3Vyc29yOjkwMA=="
Quantity of total nodes: 900
Current page 9
"Y3Vyc29yOjEwMDA="
Quantity of total nodes: 1000


In [4]:
def convertJsonToDataFrame(response_data):
    df = pd.json_normalize(response_data)
    print('Finished converting json to csv')
    return df

df = convertJsonToDataFrame(response_data)

Finished converting json to csv


In [53]:
df.head()

,stargazerCount,createdAt,name,updatedAt,url,open_issues.totalCount,closed_issues.totalCount,primaryLanguage.name,pullRequests.totalCount,releases.totalCount,primaryLanguage
0,329071,2014-12-24T17:49:19Z,freeCodeCamp,2021-08-26T12:40:01Z,https://github.com/freeCodeCamp/freeCodeCamp,15520,15397,JavaScript,15331,0,NaN
1,258404,2019-03-26T07:31:14Z,996.ICU,2021-08-26T12:20:01Z,https://github.com/996icu/996.ICU,0,0,Rust,1020,0,NaN
2,199727,2013-10-11T06:50:37Z,free-programming-books,2021-08-26T12:34:44Z,https://github.com/EbookFoundation/free-progra...,589,569,NaN,3521,0,NaN
3,190358,2016-06-06T02:34:12Z,coding-interview-university,2021-08-26T12:22:42Z,https://github.com/jwasham/coding-interview-un...,276,241,NaN,249,0,NaN
4,187341,2013-07-29T03:24:51Z,vue,2021-08-26T12:40:02Z,https://github.com/vuejs/vue,9552,9232,JavaScript,1046,210,NaN


In [5]:
def removeLastColumn(df):
    df = df.drop(labels='primaryLanguage', axis=1)
    return df
df = removeLastColumn(df)

In [6]:
df.head()

,stargazerCount,createdAt,name,updatedAt,url,open_issues.totalCount,closed_issues.totalCount,primaryLanguage.name,pullRequests.totalCount,releases.totalCount
0,329071,2014-12-24T17:49:19Z,freeCodeCamp,2021-08-26T12:40:01Z,https://github.com/freeCodeCamp/freeCodeCamp,15520,15397,JavaScript,15331,0
1,258405,2019-03-26T07:31:14Z,996.ICU,2021-08-26T12:59:36Z,https://github.com/996icu/996.ICU,0,0,Rust,1020,0
2,199726,2013-10-11T06:50:37Z,free-programming-books,2021-08-26T12:55:36Z,https://github.com/EbookFoundation/free-progra...,589,569,NaN,3524,0
3,190358,2016-06-06T02:34:12Z,coding-interview-university,2021-08-26T12:22:42Z,https://github.com/jwasham/coding-interview-un...,276,241,NaN,249,0
4,187342,2013-07-29T03:24:51Z,vue,2021-08-26T13:01:38Z,https://github.com/vuejs/vue,9552,9232,JavaScript,1046,210


In [7]:
def convertDateToAgeInDays(createdAt):
    current_date = date.today()
    createdAt = datetime.fromisoformat(createdAt.replace('Z', '+00:00')).date()
    return (current_date - createdAt).days
    
df['age'] = df['createdAt'].apply(convertDateToAgeInDays)
df['lastUpdate'] = df['updatedAt'].apply(convertDateToAgeInDays)

In [51]:
df.head(100)

,stargazerCount,createdAt,name,updatedAt,url,open_issues.totalCount,closed_issues.totalCount,primaryLanguage.name,pullRequests.totalCount,releases.totalCount,age,lastUpdate
0,329071,2014-12-24T17:49:19Z,freeCodeCamp,2021-08-26T12:40:01Z,https://github.com/freeCodeCamp/freeCodeCamp,15520,15397,JavaScript,15331,0,2437,0
1,258404,2019-03-26T07:31:14Z,996.ICU,2021-08-26T12:20:01Z,https://github.com/996icu/996.ICU,0,0,Rust,1020,0,884,0
2,199727,2013-10-11T06:50:37Z,free-programming-books,2021-08-26T12:34:44Z,https://github.com/EbookFoundation/free-progra...,589,569,NaN,3521,0,2876,0
3,190358,2016-06-06T02:34:12Z,coding-interview-university,2021-08-26T12:22:42Z,https://github.com/jwasham/coding-interview-un...,276,241,NaN,249,0,1907,0
4,187341,2013-07-29T03:24:51Z,vue,2021-08-26T12:40:02Z,https://github.com/vuejs/vue,9552,9232,JavaScript,1046,210,2950,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,52826,2013-01-14T07:54:16Z,shadowsocks-windows,2021-08-26T10:56:00Z,https://github.com/shadowsocks/shadowsocks-win...,2706,2668,C#,277,54,3146,0
96,52229,2015-03-28T00:35:42Z,keras,2021-08-26T08:55:29Z,https://github.com/keras-team/keras,10736,10362,Python,2489,28,2343,0
97,51963,2018-04-04T01:11:44Z,mall,2021-08-26T12:43:12Z,https://github.com/macrozheng/mall,400,381,Java,0,1,1240,0
98,51756,2012-02-25T12:39:13Z,httpie,2021-08-26T09:24:10Z,https://github.com/httpie/httpie,691,571,Python,194,36,3470,0


In [57]:
def convertDataFrameToCsv(response_data):
    print('Dataframe converted to CSV')
    df.to_csv('pandas.csv')

convertDataFrameToCsv(df)

Dataframe converted to CSV
